## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-10-51-17 +0000,nypost,Fitness influencer and mom Nicole Brenda Santo...,https://nypost.com/2025/08/21/world-news/fitne...
1,2025-08-21-10-47-04 +0000,bbc,Russia launches biggest wave of strikes on Ukr...,https://www.bbc.com/news/articles/c62wj8yje2eo...
2,2025-08-21-10-47-00 +0000,wsj,President Trump’s Policies Could Weaken Dollar...,https://www.wsj.com/finance/currencies/indian-...
3,2025-08-21-10-46-21 +0000,bbc,Tesco meal deal price rises 25p to £3.85,https://www.bbc.com/news/articles/clyr7renp51o...
4,2025-08-21-10-46-09 +0000,bbc,Woman jailed for race hate post released from ...,https://www.bbc.com/news/articles/c5yl7p4l11po...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,47
111,new,25
18,ukraine,14
13,russia,13
144,fire,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
194,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,119
121,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...,93
169,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,91
131,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...,91
224,2025-08-20-18-59-56 +0000,nypost,"Trump plays DJ on White House’s new patio, bla...",https://nypost.com/2025/08/20/us-news/trump-pl...,89


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
194,119,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
9,50,2025-08-21-10-21-09 +0000,nypost,Russia’s third largest aerial attack on Ukrain...,https://nypost.com/2025/08/21/world-news/major...
121,46,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...
127,46,2025-08-21-00-42-34 +0000,nypost,Supermarket chain Meijer under fire after mana...,https://nypost.com/2025/08/20/us-news/hundreds...
304,43,2025-08-20-14-01-00 +0000,wsj,Billionaire Bill Ackman has a new fascination:...,https://www.wsj.com/us-news/education/bill-ack...
184,43,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...
240,40,2025-08-20-18-09-26 +0000,nypost,"Israel calls up 60,000 reservists for looming ...",https://nypost.com/2025/08/20/world-news/israe...
131,38,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...
217,33,2025-08-20-19-42-00 +0000,nypost,Obama says Newsom’s Calif. redistricting plan ...,https://nypost.com/2025/08/20/us-news/obama-sa...
169,33,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
